[View in Colaboratory](https://colab.research.google.com/github/joheras/CLoDSA/blob/master/notebooks/CLODSA_Keras.ipynb)

# Online augmentation for Keras

In this notebook, we illustrate how CLODSA can be employed for online augmentation in Keras using a subset of the [cats and dogs dataset](https://www.kaggle.com/c/dogs-vs-cats). Such a subset can be downloaded using the following command.

In [ ]:
!wget https://www.dropbox.com/s/dvqk6ukk3sli72t/cats_dogs.zip?dl=0 -O cats_dogs.zip
!unzip cats_dogs.zip

We can check the amount of images in each one of the folders.

In [ ]:
print("Number of images in cats folder")
!ls cats_dogs/cats/ | wc -l
print("Number of images in dogs folder")
!ls cats_dogs/dogs/ | wc -l

## Augmentation techniques

We will use the following techniques for online data augmentation:
- Vertical and horizontal flip.
- Equalize histogram
- Median blur
- Salt and pepper noise
- Crops


## Installing the necessary libraries

In case that CLODSA is not installed in your system, the first task consists in installing it using ``pip``.

In [ ]:
!pip install clodsa

## Loading the necessary libraries

The first step in the pipeline consists in loading the necessary libraries to apply the data augmentation techniques in CLODSA and to train a Keras model. 

In [ ]:
from matplotlib import pyplot as plt
from clodsa.augmentors.augmentorFactory import createAugmentor
from clodsa.transformers.transformerFactory import transformerGenerator
from clodsa.techniques.techniqueFactory import createTechnique
from keras.optimizers import SGD
from clodsa.utils.minivgg import MiniVGGNet
import cv2
%matplotlib inline

## Creating the augmentor object

As explained in the documentation of CLODSA, we need to specify some parameters for the augmentation process, and use them to create an augmentor object.  

_The kind of problem_. In this case, we are working in a classification problem

In [ ]:
PROBLEM = "classification"

_The annotation mode_. The annotation mode is folders. 

In [ ]:
ANNOTATION_MODE = "folders"

_The input path_. The input path containing the images. 

In [ ]:
INPUT_PATH = "cats_dogs/"

_The generation mode_. In this case, linear, that is, all the augmentation techniques are applied to all the images of the original dataset. 

In [ ]:
GENERATION_MODE = "linear"

_The output mode_. The generated images will be provided to train a Keras model.  

In [ ]:
OUTPUT_MODE = "keras"

We must also fix other parameters.

In [ ]:
PARAMETERS = {
    "batchSize": 32,
    "width": 64,
    "height": 64
  }

Using the above information, we can create our augmentor object. 

In [ ]:
augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,PARAMETERS)

## Adding the augmentation techniques

Now, we define the techniques that will be applied in our augmentation process and add them to our augmentor object. To illustrate the transformations, we will use the following image of the dataset. 



First of all, we must define a transformer generator.

In [ ]:
transformer = transformerGenerator(PROBLEM)

#### Vertical flip

In [ ]:
vFlip = createTechnique("flip",{"flip":0})
augmentor.addTransformer(transformer(vFlip))

#### Horizontal flip

In [ ]:
hFlip = createTechnique("flip",{"flip":1})
augmentor.addTransformer(transformer(hFlip))

#### Crop

In [ ]:
crop = createTechnique("crop",{"percentage":0.9,"startFrom": "TOPLEFT"})
augmentor.addTransformer(transformer(crop))

**Equalize histogram**

In [ ]:
equalize = createTechnique("equalize_histogram",{})
augmentor.addTransformer(transformer(equalize))

**Median blur**

In [ ]:
median = createTechnique("median_blur",{"kernel":3})
augmentor.addTransformer(transformer(median))

**Salt and pepper**

In [ ]:
salt_and_pepper = createTechnique("salt_and_pepper",{"low":0,"high":255})
augmentor.addTransformer(transformer(salt_and_pepper))

#### None
(to keep also the original image)

In [ ]:
none = createTechnique("none",{})
augmentor.addTransformer(transformer(none))

## Definition of the model

Now we define the model using a version of the VGG network.

In [ ]:
opt = SGD(lr=0.05)
model = MiniVGGNet.build(width=PARAMETERS["width"], height=PARAMETERS["height"], depth=3)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

## Training the model

Finally, we can train the model.

In [ ]:
H = model.fit_generator(augmentor.applyAugmentation(),
                         steps_per_epoch= PARAMETERS["batchSize"],
                         epochs=100, verbose=1)